# NN(Neural network model) : Multi-layer perceptron (MLP)
> * I try Neural network model "multi-layer Perceptron classifier".
> * This is the advanced models of Deep Learning.
> * I hope this will help you.Please excuse if there are any mistakes🙏

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# faster downloading
import datatable as dt
train = dt.fread(r'/kaggle/input/tabular-playground-series-nov-2021/train.csv').to_pandas()
test = dt.fread(r'/kaggle/input/tabular-playground-series-nov-2021/test.csv').to_pandas()

In [ ]:
# Reduce memory usage if PC is 
# train features
for col in train.columns:
    if train[col].dtype == "float64":
        train[col]=pd.to_numeric(train[col], downcast="float")
    if train[col].dtype == "int64":
        train[col]=pd.to_numeric(train[col], downcast="integer")
    

# test features
for col in test.columns:
    if test[col].dtype == "float64":
        test[col]=pd.to_numeric(test[col], downcast="float")
    if train[col].dtype == "int64":
        test[col]=pd.to_numeric(test[col], downcast="integer")  

In [ ]:
id = test['id'].copy()
train = train.drop('id' ,axis = 1)
test = test.drop('id', axis = 1)

In [ ]:
X = train.drop('target', axis = 1).copy()
y = train['target'].copy()
X_test = test.copy()

del train
del test

---------------


# improved tips💡
> * This is basic model. so you may
> * add parameters like this -> (max_iter = 1000, hidden_layer_sizes = (10, 10), alpha = 0.0001, solver = 'adam', random_state = 42)
> * using scaler, adding or choosing feature, more folds, random_state etc...Good luck!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, auc

In [ ]:
import time
start = time.perf_counter()
print("--------⏱start-----------")

In [ ]:
splits = 5
seed = 42
skf = StratifiedKFold(n_splits = splits, shuffle=True, random_state=seed)

scores = []
pred= []

for fold, (idx_train, idx_valid) in enumerate(skf.split(X, y)):
    
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = MLPClassifier()
    model.fit(X_train, y_train)
    
    pred_valid = model.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)

    test_preds = model.predict_proba(X_test)[:,1]
    pred.append(test_preds)
    print("fold :  ",fold, " MLPsore : ", score)

In [ ]:
p_1 = sum(scores)/splits
print(p_1)
print(scores)

In [ ]:
time_end = time.perf_counter()
process_time = time_end - start
print("process_time:{0}".format(process_time) )

In [ ]:
predictions = np.mean(np.column_stack(pred) ,axis = 1)
submission = pd.DataFrame({'id' : id, 'target' : predictions})
submission.to_csv("submission.csv", index = False)

submission.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.kdeplot(submission['target'],fill=True, color = "#20B2AA")
plt.title('MLPClassifier ')